# Define mock time series dataset

In [1]:
from pandas import DataFrame

df = DataFrame()
df['t'] = [x for x in range(10)]
print(df)

   t
0  0
1  1
2  2
3  3
4  4
5  5
6  6
7  7
8  8
9  9


# Shift observations down by one time step

In [2]:
df['t+1'] = df['t'].shift(-1)
print(df)

   t  t+1
0  0  1.0
1  1  2.0
2  2  3.0
3  3  4.0
4  4  5.0
5  5  6.0
6  6  7.0
7  7  8.0
8  8  9.0
9  9  NaN


# Define series to supervised function

In [3]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset
    Arguments:
        data: Sequence of observations as a list or NumPy array
        n_in: Number of lag observations as input (X)
        n_out: Number of observations as output
        dropnan: Boolean whether or not to drop rows with NaN values
    Returns:
        Pandas DataFrame of series framed for supervised learning
    """
    
    # getting the number of columns
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df = DataFrame(data)
    cols, names = list(),list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ..., t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i==0:
            names += [('var%d(t+%d)' % (j+1,i)) for j in range(n_vars)]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


# Test series to supervised function

In [4]:
import numpy as np

data1 = np.array([[0.1,100],[0.2, 200],[0.3,300],[0.4,400],[0.5,500],[0.6,600],[0.7,700],[0.8,800],[0.9,900]])

reframed_data1_v1 = series_to_supervised(data1, 1)
reframed_data1_v2 = series_to_supervised(data1, 2)

print(DataFrame(data1))
print(reframed_data1_v1)
print(reframed_data1_v2)

     0      1
0  0.1  100.0
1  0.2  200.0
2  0.3  300.0
3  0.4  400.0
4  0.5  500.0
5  0.6  600.0
6  0.7  700.0
7  0.8  800.0
8  0.9  900.0
   var1(t-1)  var2(t-1)  var1(t+0)  var2(t+0)
1        0.1      100.0        0.2      200.0
2        0.2      200.0        0.3      300.0
3        0.3      300.0        0.4      400.0
4        0.4      400.0        0.5      500.0
5        0.5      500.0        0.6      600.0
6        0.6      600.0        0.7      700.0
7        0.7      700.0        0.8      800.0
8        0.8      800.0        0.9      900.0
   var1(t-2)  var2(t-2)  var1(t-1)  var2(t-1)  var1(t+0)  var2(t+0)
2        0.1      100.0        0.2      200.0        0.3      300.0
3        0.2      200.0        0.3      300.0        0.4      400.0
4        0.3      300.0        0.4      400.0        0.5      500.0
5        0.4      400.0        0.5      500.0        0.6      600.0
6        0.5      500.0        0.6      600.0        0.7      700.0
7        0.6      600.0        0.7    